In [38]:
import pandas as pd
import os
from data_utils import get_raw_dataset

In [2]:
!ls datos_series_temporales

10-year-treasury-bond-rate-yield-chart.csv  IVE_tickbidask.txt
equity_trades_only_1min.csv		    OIH_adjusted.txt
IBM_adjusted.txt			    OIH_unadjusted.txt
IBM_unadjusted.txt			    WDC_tickbidask.txt


In [3]:
!head datos_series_temporales/IVE_tickbidask.txt

In [7]:
!head -10 datos_series_temporales/OIH_adjusted.txt

In [103]:
def _read_sp500(file):
    df =  pd.read_csv(file, names = ["date", "time", "price", "bid", "ask", "volume"], parse_dates=[["date", "time"]])
    return df

In [140]:
def _read_oil(file):
    df = pd.read_csv(
        file,
        names=["date", "time", "open", "high", "low", "close", "volume"],
        parse_dates=[["date", "time"]]
    )
    return df

In [21]:
sp500 = _read_sp500("datos_series_temporales/IVE_tickbidask.txt")

In [141]:
oil = _read_oil("datos_series_temporales/OIH_adjusted.txt")

In [22]:
sp500.shape

(8626971, 5)

In [142]:
oil.shape

(1815874, 6)

In [143]:
oil.head()

,date_time,open,high,low,close,volume
0,2001-02-26 09:41:00,404.82,404.82,404.82,404.82,4377
1,2001-02-26 09:44:00,406.24,406.24,406.24,406.24,106
2,2001-02-26 09:46:00,407.89,407.89,407.89,407.89,106
3,2001-02-26 10:14:00,411.91,411.91,411.91,411.91,42
4,2001-02-26 10:18:00,414.28,414.28,414.28,414.28,6344


In [150]:
def cut_oil(oil_df, assetsdf):
    first_date = assetsdf.index[0]
    last_date = assetsdf.index[-1]
    after_first = oil_df["date_time"] >= first_date
    before_last = oil_df["date_time"] <= last_date
    between = after_first & before_last
    oil_df = oil_df.loc[between]
    oil_df.columns = ["date"] + [f"oil_{col}" for col in oil_df.columns if "date" not in col]
    oil_df = oil_df.resample("H", on="date").aggregate("mean")
    oil_df.fillna(method="ffill", inplace=True)
    return oil_df

In [151]:
oil_cut = cut_oil(oil, assets_df)

In [152]:
oil_cut.head()

,oil_open,oil_high,oil_low,oil_close,oil_volume
date,,,,,
2018-12-19 08:00:00,289.180000,289.180000,289.180000,289.180000,53.000000
2018-12-19 09:00:00,288.941429,289.236000,288.683143,288.952286,1376.628571
2018-12-19 10:00:00,291.335000,291.529000,291.116833,291.336000,2281.416667
2018-12-19 11:00:00,290.732500,290.856333,290.559500,290.720667,1254.316667
2018-12-19 12:00:00,291.526500,291.634333,291.431833,291.529500,1390.316667


In [25]:
df = sp500.set_index("date_time")

In [26]:
df.resample("m").aggregate("mean")

,price,bid,ask,volume
date_time,,,,
2009-09-30,51.377042,51.368301,51.384011,233.884076
2009-10-31,51.348281,51.340591,51.356182,226.439439
2009-11-30,51.528722,51.519520,51.537647,254.567487
2009-12-31,53.236331,53.227453,53.243479,289.358402
2010-01-31,53.916272,53.910540,53.922525,220.264727
...,...,...,...,...
2020-10-31,114.078825,114.067478,114.085537,211.418889
2020-11-30,121.259634,121.245374,121.276746,246.063958
2020-12-31,126.693219,126.681529,126.702783,230.108560


In [27]:
times = pd.to_datetime(df.index)

In [28]:
times[0]

Timestamp('2009-09-28 09:30:00')

In [29]:
df.groupby([times.day, times.hour, times.minute])["price"].mean()


date_time  date_time  date_time
1          6          43            53.360000
           7          0            110.500000
                      2            117.560000
                      39            61.150000
           8          8             60.445000
                                      ...    
31         18         25           109.650000
                      30           111.424348
           19         55           116.210000
           20         0            111.424348
                      12           118.984500
Name: price, Length: 13466, dtype: float64

In [32]:
df_minute = sp500.groupby(sp500.date_time.dt.time)[["price", "bid", "ask", "volume"]].mean()

In [33]:
df_minute.head()

,price,bid,ask,volume
date_time,,,,
04:00:00,112.905,57.74,55.165,100.0
04:00:36,108.950,103.95,108.950,237.0
04:08:09,115.060,0.00,115.060,238.0
04:13:00,110.000,109.40,110.000,100.0
04:23:13,100.500,99.49,0.000,100.0


In [34]:
df_minute.shape

(25035, 4)

In [35]:
df_hour = df.resample("H").aggregate("mean")

In [36]:
df_hour.shape

(99492, 4)

In [37]:
df_hour.head()

,price,bid,ask,volume
date_time,,,,
2009-09-28 09:00:00,51.048208,51.037449,51.054184,303.945578
2009-09-28 10:00:00,51.277057,51.267413,51.282441,304.741348
2009-09-28 11:00:00,51.360257,51.352055,51.364887,344.380952
2009-09-28 12:00:00,51.460638,51.452312,51.465538,219.145161
2009-09-28 13:00:00,51.498606,51.488495,51.503692,236.333333


In [68]:
assets_df = get_raw_dataset("trainTimeSeries/trainTimeSeries/TrainCandles")

100%|██████████| 96/96 [00:00<00:00, 4399.76it/s]


In [69]:
assets_df.shape

(9288, 96)

In [70]:
assets_df = assets_df.resample("H").aggregate("mean")

In [72]:
assets_df.shape

(13127, 96)

In [73]:
assets_df.fillna(method="ffill", inplace=True)

In [74]:
assets_df.index[0]

Timestamp('2018-12-19 01:00:00', freq='H')

In [127]:
sp500.columns

Index(['date_time', 'price', 'bid', 'ask', 'volume'], dtype='object')

In [133]:
def cut_sp500(df_sp, assetsdf):
    first_date = assetsdf.index[0]
    last_date = assetsdf.index[-1]
    after_first = df_sp["date_time"] >= first_date
    before_last = df_sp["date_time"] <= last_date
    between = after_first & before_last
    df = df_sp.loc[between]
    #df.set_index("date_time", inplace=True)
    df.columns = ["date"] + [f"sp500_{col}" for col in ["price", "bid", "ask", "volume"]]
    df = df.resample("H", on="date").aggregate("mean")
    df.fillna(method="ffill", inplace=True)
    return df

In [134]:
sp500_cutted = cut_sp500(sp500, assets_df)

In [135]:
sp500_cutted.columns

Index(['sp500_price', 'sp500_bid', 'sp500_ask', 'sp500_volume'], dtype='object')

In [136]:
merged_df = pd.merge(assets_df, sp500_cutted, on="date", how="outer")

In [137]:
sp500_cutted.shape


(13116, 4)

In [138]:
merged_df.shape

(13127, 100)

In [139]:
merged_df.isna().sum()

ZVQ_close          0
NCT_close         21
YAX_close       8853
OOS_close       1629
GFJ_close       3476
                ... 
RWJ_close       1125
sp500_price       11
sp500_bid         11
sp500_ask         11
sp500_volume      11
Length: 100, dtype: int64

In [120]:
assets_df.head()

,ZVQ_close,NCT_close,YAX_close,OOS_close,GFJ_close,FIR_close,USX_close,FSK_close,TMF_close,TDD_close,...,AZG_close,OJG_close,WWT_close,BOT_close,TXR_close,RAT_close,DIG_close,SRI_close,PHI_close,RWJ_close
date,,,,,,,,,,,,,,,,,,,,,
2018-12-19 01:00:00,519.23,NaN,NaN,NaN,NaN,NaN,NaN,154.69,NaN,116.07,...,129.87,NaN,129.98,NaN,NaN,140.36,NaN,92.19,NaN,NaN
2018-12-19 02:00:00,519.63,NaN,NaN,NaN,NaN,NaN,NaN,155.01,NaN,116.25,...,129.95,NaN,130.02,NaN,NaN,140.55,NaN,92.11,NaN,NaN
2018-12-19 03:00:00,519.95,NaN,NaN,NaN,NaN,NaN,NaN,155.08,NaN,116.45,...,130.07,NaN,130.10,NaN,NaN,140.51,NaN,92.12,NaN,NaN
2018-12-19 04:00:00,520.09,NaN,NaN,NaN,NaN,NaN,NaN,155.16,NaN,116.65,...,130.26,NaN,130.50,NaN,NaN,140.57,NaN,92.18,NaN,NaN
2018-12-19 05:00:00,520.06,NaN,NaN,NaN,NaN,NaN,NaN,155.23,NaN,116.30,...,130.07,NaN,130.28,NaN,NaN,140.50,NaN,92.07,NaN,NaN


In [118]:
assets_df.shape

(13127, 96)

In [119]:
[idx for idx in assets_df.index if idx not in sp500_cutted.index]

[Timestamp('2018-12-19 01:00:00', freq='H'),
 Timestamp('2018-12-19 02:00:00', freq='H'),
 Timestamp('2018-12-19 03:00:00', freq='H'),
 Timestamp('2018-12-19 04:00:00', freq='H'),
 Timestamp('2018-12-19 05:00:00', freq='H'),
 Timestamp('2018-12-19 06:00:00', freq='H'),
 Timestamp('2018-12-19 07:00:00', freq='H'),
 Timestamp('2018-12-19 08:00:00', freq='H'),
 Timestamp('2020-06-17 21:00:00', freq='H'),
 Timestamp('2020-06-17 22:00:00', freq='H'),
 Timestamp('2020-06-17 23:00:00', freq='H')]

In [78]:
df_hour.loc['2009-10-19 17:00:00']

price    NaN
bid      NaN
ask      NaN
volume   NaN
Name: 2009-10-19 17:00:00, dtype: float64

In [55]:
str(assets_df.index[0])

'2018-12-19 01:00:00'

In [63]:
df_hour.index

DatetimeIndex(['2009-09-28 09:00:00', '2009-09-28 10:00:00',
               '2009-09-28 11:00:00', '2009-09-28 12:00:00',
               '2009-09-28 13:00:00', '2009-09-28 14:00:00',
               '2009-09-28 15:00:00', '2009-09-28 16:00:00',
               '2009-09-28 17:00:00', '2009-09-28 18:00:00',
               ...
               '2021-02-02 11:00:00', '2021-02-02 12:00:00',
               '2021-02-02 13:00:00', '2021-02-02 14:00:00',
               '2021-02-02 15:00:00', '2021-02-02 16:00:00',
               '2021-02-02 17:00:00', '2021-02-02 18:00:00',
               '2021-02-02 19:00:00', '2021-02-02 20:00:00'],
              dtype='datetime64[ns]', name='date_time', length=99492, freq='H')

In [64]:
assets_df.index

DatetimeIndex(['2018-12-19 01:00:00', '2018-12-19 02:00:00',
               '2018-12-19 03:00:00', '2018-12-19 04:00:00',
               '2018-12-19 05:00:00', '2018-12-19 06:00:00',
               '2018-12-19 07:00:00', '2018-12-19 08:00:00',
               '2018-12-19 09:00:00', '2018-12-19 10:00:00',
               ...
               '2019-01-02 00:00:00', '2019-01-02 01:00:00',
               '2019-01-02 02:00:00', '2019-01-02 03:00:00',
               '2019-01-02 04:00:00', '2019-01-02 05:00:00',
               '2019-01-02 06:00:00', '2019-01-02 07:00:00',
               '2019-01-02 08:00:00', '2018-12-24 21:00:00'],
              dtype='datetime64[ns]', name='date', length=9288, freq=None)

In [79]:
df_hour

,price,bid,ask,volume
date_time,,,,
2009-09-28 09:00:00,51.048208,51.037449,51.054184,303.945578
2009-09-28 10:00:00,51.277057,51.267413,51.282441,304.741348
2009-09-28 11:00:00,51.360257,51.352055,51.364887,344.380952
2009-09-28 12:00:00,51.460638,51.452312,51.465538,219.145161
2009-09-28 13:00:00,51.498606,51.488495,51.503692,236.333333
...,...,...,...,...
2021-02-02 16:00:00,128.882500,128.772500,129.762500,11063.000000
2021-02-02 17:00:00,NaN,NaN,NaN,NaN
2021-02-02 18:00:00,128.890000,128.570000,129.870000,0.000000


In [83]:
df_hour.loc["2019-01-04 00:00:00"]

price    NaN
bid      NaN
ask      NaN
volume   NaN
Name: 2019-01-04 00:00:00, dtype: float64

In [84]:
df_hour.loc["2018-01-01 00:00:00":"2020-02-02 00:00:00"]

,price,bid,ask,volume
date_time,,,,
2018-01-01 00:00:00,NaN,NaN,NaN,NaN
2018-01-01 01:00:00,NaN,NaN,NaN,NaN
2018-01-01 02:00:00,NaN,NaN,NaN,NaN
2018-01-01 03:00:00,NaN,NaN,NaN,NaN
2018-01-01 04:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2020-02-01 20:00:00,NaN,NaN,NaN,NaN
2020-02-01 21:00:00,NaN,NaN,NaN,NaN
2020-02-01 22:00:00,NaN,NaN,NaN,NaN


In [85]:
df_hour.isna().sum()

price     74378
bid       74378
ask       74378
volume    74378
dtype: int64

In [86]:
sp500.isna().sum()

date_time    0
price        0
bid          0
ask          0
volume       0
dtype: int64

In [88]:
sp500.loc["2018-01-01 00:00:00":"2020-02-02 00:00:00"]

,date_time,price,bid,ask,volume
2018000,2013-12-27 11:24:06,85.1100,85.10,85.11,100
2018001,2013-12-27 11:24:23,85.0907,85.09,85.10,173
2018002,2013-12-27 11:24:37,85.0916,85.08,85.10,291
2018003,2013-12-27 11:24:43,85.0900,85.08,85.10,200
2018004,2013-12-27 11:24:43,85.0900,85.08,85.10,100
...,...,...,...,...,...
2019995,2013-12-31 12:40:06,85.4200,85.42,85.43,280
2019996,2013-12-31 12:40:06,85.4200,85.41,85.42,100
2019997,2013-12-31 12:40:19,85.4100,85.40,85.41,100
2019998,2013-12-31 12:40:19,85.4100,85.40,85.41,100


In [89]:
sp500.shape

(8626971, 5)

In [96]:
sp500[6000000:7000000]

,date_time,price,bid,ask,volume
6000000,2018-10-08 09:39:36,116.21,116.19,116.21,500
6000001,2018-10-08 09:39:36,116.21,116.19,116.21,100
6000002,2018-10-08 09:39:36,116.21,116.19,116.21,500
6000003,2018-10-08 09:39:36,116.21,116.19,116.21,500
6000004,2018-10-08 09:39:37,116.21,116.20,116.21,500
...,...,...,...,...,...
6999995,2019-09-09 15:36:29,118.92,118.91,118.93,100
6999996,2019-09-09 15:36:29,118.92,118.91,118.93,100
6999997,2019-09-09 15:37:05,118.91,118.90,118.91,1800
6999998,2019-09-09 15:37:05,118.91,118.90,118.91,200


In [97]:
sp500.loc["2018-10-08 09:39:36":"2019-09-09 15:37:05"]

,date_time,price,bid,ask,volume
2018000,2013-12-27 11:24:06,85.1100,85.10,85.11,100
2018001,2013-12-27 11:24:23,85.0907,85.09,85.10,173
2018002,2013-12-27 11:24:37,85.0916,85.08,85.10,291
2018003,2013-12-27 11:24:43,85.0900,85.08,85.10,200
2018004,2013-12-27 11:24:43,85.0900,85.08,85.10,100
...,...,...,...,...,...
2018995,2013-12-30 13:59:52,85.1299,85.12,85.13,150
2018996,2013-12-30 14:00:51,85.1200,85.11,85.12,300
2018997,2013-12-30 14:00:51,85.1199,85.11,85.12,300
2018998,2013-12-30 14:00:51,85.1199,85.11,85.12,350


In [99]:
after_start_date = sp500["date_time"] > "2018-10-08 09:39:36"
before_end_date = sp500["date_time"] < "2019-09-09 15:37:05"

In [101]:
between_two_dates = after_start_date & before_end_date
filtered_dates = sp500.loc[between_two_dates]

In [102]:
filtered_dates

,date_time,price,bid,ask,volume
6000004,2018-10-08 09:39:37,116.2100,116.20,116.21,500
6000005,2018-10-08 09:39:37,116.2100,116.20,116.21,500
6000006,2018-10-08 09:39:37,116.2100,116.20,116.21,500
6000007,2018-10-08 09:39:37,116.2100,116.20,116.21,500
6000008,2018-10-08 09:39:37,116.2100,116.20,116.21,500
...,...,...,...,...,...
6999992,2019-09-09 15:36:12,118.9100,118.91,118.92,100
6999993,2019-09-09 15:36:12,118.9100,118.91,118.92,100
6999994,2019-09-09 15:36:16,118.9064,118.90,118.92,285
6999995,2019-09-09 15:36:29,118.9200,118.91,118.93,100
